In [10]:
import numpy as np
import pandas as pd
import os

In [24]:
dataLocation="../Data/KaggleData/US_Funds_dataset_from_Yahoo_Finance/ETF prices.csv"
rawData=pd.read_csv(dataLocation).drop(columns=["adj_close","price_date"])
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3866030 entries, 0 to 3866029
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   fund_symbol  object 
 1   open         float64
 2   high         float64
 3   low          float64
 4   close        float64
 5   volume       int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 177.0+ MB


In [41]:
# stocks=rawData["fund_symbol"].unique()
# it takes abt 5 mins to do this find another way

# dfs= [0]* stocks.shape[0]
# for i,stock in enumerate(stocks):
#     print(i)
#     dfs[i]=rawData[rawData["fund_symbol"] == stock]
# rawData=None

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198


KeyboardInterrupt: 

In [4]:
lookback_window=10 #I don't have any idea which number is best
def create_sequences(data, lookback):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i + lookback])
        y.append(data[i + lookback])
    return np.array(X), np.array(y)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


num_features=5 #for OHLC and volume
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(lookback_window, num_features)))
model.add(LSTM(64))
model.add(Dense(1))  # Predicting one value, like 'close'
model.compile(optimizer='adam', loss='mean_squared_error')